In [1]:
import torch
from models import BERT_Arch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import shap

In [2]:
device = 'cuda'
pre_trained_model_name = 'seyonec/ChemBERTa_zi' \
                         'nc250k_v2_40k'
bert_tokenizer = AutoTokenizer.from_pretrained(pre_trained_model_name)
bert_model = AutoModel.from_pretrained(pre_trained_model_name, return_dict=True).to(device)

batch_size = 256

model = BERT_Arch(bert_model).to(device)
ckpt = torch.load('res/best_fine_tune_ChemBERTa_task1_seed_1.pt')
model.load_state_dict(ckpt)
model.eval()

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /seyonec/ChemBERTa_zinc250k_v2_40k/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000028D75935510>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/seyonec/ChemBERTa_zinc250k_v2_40k/resolve/main/tokenizer_config.json
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /seyonec/ChemBERTa_zinc250k_v2_40k/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000028D75A16860>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/seyonec/ChemBERTa_zinc250k_v2_40k/resolve/main/config.json
Some weights of the model checkpoint at seyonec/ChemBERTa_zinc250k_v2_40k were not used when initializing RobertaModel: ['lm_head.decoder.

BERT_Arch(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [3]:
def f(x):
    tokens= bert_tokenizer.batch_encode_plus(
        x.tolist(),
        max_length=80,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )
    # print(tokens)
    input_ids = torch.tensor(tokens['input_ids']).to(device)
    att_masks = torch.tensor(tokens['attention_mask']).to(device)
    outputs = model(input_ids, att_masks).detach().cpu().numpy()
    return outputs

In [4]:
explainer = shap.Explainer(f, bert_tokenizer)

df = pd.read_csv('data/bTox_test.csv').to_numpy()
canonicals = df[:, 0]


In [5]:
shap_vals = explainer(canonicals, fixed_context=1)


The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or us

In [6]:
shap.plots.text(shap_vals[:10])